# This app focuses on extracting cryptocurrencies data from coingecko's API

## APIs used:

    - Get/coins/markets to get a list of all supported coins, ath, price, volume and other info
    - Get/coins/{id}/history to get community and developer data


In [43]:
import requests
import pandas as pd
import time

In [2]:
# Create an empty list to hold the coins information
crypto_list = []

In [3]:
# Find an API that has information on coin

# Make a loop to get 10 pages
for i in range(1,10,1):
    page_number = i
    base_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page={page_number}&sparkline=false"
    print(base_url)

    # Request json file from the url
    response_json = requests.get(base_url).json()

    # append response to empty list
    crypto_list.append(response_json)

https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=1&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=2&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=3&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=4&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=5&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=6&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=7&sparkline=false
https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=8&sparkline=false
https://api.coingecko.com/api/v3

In [4]:
# Add all the data frames together
# Set up the a value for the previous dataframe
previous_df = pd.DataFrame(crypto_list[0])
for i in range(1, len(crypto_list)):
    next_df = pd.DataFrame(crypto_list[i])
    complete_df = pd.concat([previous_df, next_df])
    previous_df = complete_df



In [5]:
clean_complete_df = complete_df.reset_index(drop=True)
#clean_complete_df

In [6]:
# See the name of the columns
#clean_complete_df.columns

In [7]:
# Only get specific columns
crypto_df = clean_complete_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]

In [8]:
# Only run this cell if the data hasn't been renamed and cleaned
if crypto_df.columns[3] == "market_cap":
    # Clean data by dropping columns, ONLY RUN IT ONCE, since columns will change name
    #crypto_df = crypto_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]
    crypto_df['market_cap'] = crypto_df['market_cap']/1000000000

    # Rename columns to see easier
    crypto_df.rename(columns={'market_cap':'market_cap_billion(2021)',
                              'current_price' : 'price_04-22-21'

                             }, inplace=True)


<ipython-input-8-3d38269c4a46>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crypto_df['market_cap'] = crypto_df['market_cap']/1000000000
/Users/johannvillalvir/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
# Verify data drame
crypto_df.head()

,id,symbol,price_04-22-21,market_cap_billion(2021),market_cap_rank,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date
0,bitcoin,btc,53614.00,1001.972785,1,64805.00,-17.50247,2021-04-14T11:54:46.763Z,67.810000,7.874247e+04,2013-07-06T00:00:00.000Z
1,ethereum,eth,2490.28,287.902762,2,2640.37,-6.07681,2021-04-22T16:34:50.091Z,0.432979,5.726573e+05,2015-10-20T00:00:00.000Z
2,binancecoin,bnb,535.06,82.624517,3,610.06,-12.48478,2021-04-12T09:32:32.635Z,0.039818,1.340739e+06,2017-10-19T00:00:00.000Z
3,ripple,xrp,1.24,56.745919,4,3.40,-63.79962,2018-01-07T00:00:00.000Z,0.002686,4.569885e+04,2014-05-22T00:00:00.000Z
4,tether,usdt,1.00,50.166901,5,1.32,-24.39155,2018-07-24T00:00:00.000Z,0.572521,7.473102e+01,2015-03-02T00:00:00.000Z


In [10]:
# Save data as csv
crypto_df.to_csv("data/cryptocurrencies_data_2021.csv")

In [11]:
# Get each coin's price on 01-04-2018 to now
# Make empty list to hold the value of coins

coin_data_2018 = []

target_date = "01-04-2018"

In [59]:
# Set up timer to slow down requests speed
def timer_milliseconds(milliseconds = 3):
    for i in range(0, seconds):
        print(f"Counting... {i +1}")
        time.sleep(1/10)
        

In [60]:
timer(10)

Counting... 1
Counting... 2
Counting... 3
Counting... 4
Counting... 5
Counting... 6
Counting... 7
Counting... 8
Counting... 9
Counting... 10


In [19]:
# Make function to perform request based on specific data, use dataframe above's id to search that specific coin
def obtain_metrics_by_date (target_df = crypto_df, target_list = coin_data_2018, date='01-04-2018'):
    # Set up error counter to break out if there are too many errors
    key_error_counter = 0
    #iterate through dataframe gettind index and row
    for index, row in target_df.iterrows():
        
        #get coin name through dictionary syntax
        target_coin = row['id']
        
        # print the current row's coin
        print(f"Processing {target_coin}")
        
        # number of key_errors allowed 
        if key_error_counter < 100:
            try:
                # build url
                price_url = f"https://api.coingecko.com/api/v3/coins/{target_coin}/history?date={date}localization=False"
                print(price_url)
                # save json respons
                response_json = requests.get(price_url).json()
                # traverse json getting specific data
                coin_id = response_json['id']
                coin_price = response_json['market_data']['current_price']['usd']
                coin_reddit_subscribers = response_json['community_data']['reddit_subscribers']
                coin_twitter_followers = response_json['community_data']['twitter_followers']
                coin_alexa_rank = response_json['public_interest_stats']['alexa_rank']
                # Add data to empty list in dictionary style
                target_list.append({
                    "id" : coin_id,
                    f"price {date}" : coin_price,
                    f"reddit_subscribers {date}" : coin_reddit_subscribers,
                    f"twitter_followers {date}" : coin_twitter_followers,
                    f"alexa_rank {date}" : coin_alexa_rank
                       })
            # The first error is a no market_data for that coin
            except :
                try:
                    # The first error will be a key error, since young coins dont have 'market_data'
                    # add one to the key_error counter
                    key_error_counter+=1
                    #print message
                    print("Key error. Coin too young to find all data")   
                    print(f"{target_coin}")
                    print(price_url)
                    # make a request again, in case we got timedout
                    response_json = requests.get(price_url).json()
                    #still add the coin to the list
                    target_list.append({'id': coin_id})
                 
                # the next error would be a timedout error
                except :
                    print('Ruh Roh! Looks like you have been timed out')
                    print(f"The last coin to be pulled was {target_coin} at index {index}")
                    # If we got timed out, we want to exit the function
                    break
        # print message for too many requests errors
        else:
            print(f"You have reached {key_error_counter} keyerrors.")
            print(f"Last data pulled at position index:{index}, id: {target_coin}")
            break
    print("Done with Data Retieval")
    print(f"We found a total of {len(target_list)} entries")
    print(f"{index}, {target_coin}")
  



In [14]:
obtain_metrics_by_date()

Processing bitcoin
https://api.coingecko.com/api/v3/coins/bitcoin/history?date=01-04-2018localization=False
Processing ethereum
https://api.coingecko.com/api/v3/coins/ethereum/history?date=01-04-2018localization=False
Processing binancecoin
https://api.coingecko.com/api/v3/coins/binancecoin/history?date=01-04-2018localization=False
Processing ripple
https://api.coingecko.com/api/v3/coins/ripple/history?date=01-04-2018localization=False
Processing tether
https://api.coingecko.com/api/v3/coins/tether/history?date=01-04-2018localization=False
Processing cardano
https://api.coingecko.com/api/v3/coins/cardano/history?date=01-04-2018localization=False
Processing dogecoin
https://api.coingecko.com/api/v3/coins/dogecoin/history?date=01-04-2018localization=False
Processing polkadot
https://api.coingecko.com/api/v3/coins/polkadot/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
polkadot
https://api.coingecko.com/api/v3/coins/polkadot/history?date=01-04-2018loc

Key error. Coin too young to find all data
cdai
https://api.coingecko.com/api/v3/coins/cdai/history?date=01-04-2018localization=False
Processing avalanche-2
https://api.coingecko.com/api/v3/coins/avalanche-2/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
avalanche-2
https://api.coingecko.com/api/v3/coins/avalanche-2/history?date=01-04-2018localization=False
Processing huobi-token
https://api.coingecko.com/api/v3/coins/huobi-token/history?date=01-04-2018localization=False
Processing compound-governance-token
https://api.coingecko.com/api/v3/coins/compound-governance-token/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
compound-governance-token
https://api.coingecko.com/api/v3/coins/compound-governance-token/history?date=01-04-2018localization=False
Processing thorchain
https://api.coingecko.com/api/v3/coins/thorchain/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
thorchain
h

In [25]:
len(coin_data_2018)

113

In [32]:
# If we got timed out, make a DF starting from the last token proccesed to run the function again
last_coin_proccesed = len(coin_data_2018)
left_over_df = crypto_df.iloc[last_coin_proccesed:]
left_over_df



,id,symbol,price_04-22-21,market_cap_billion(2021),market_cap_rank,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date
169,alpha-finance,alpha,1.340000,0.335534,169,2.930000,-54.07560,2021-02-05T16:37:55.153Z,0.026358,4999.78452,2020-11-03T04:33:55.723Z
170,the-sandbox,sand,0.484793,0.332967,172,0.915838,-47.91275,2021-03-28T16:04:16.964Z,0.028978,1546.21663,2020-11-04T15:59:14.441Z
171,alien-worlds,tlm,0.265910,0.329919,171,7.160000,-96.29005,2021-04-11T06:36:48.412Z,0.199118,33.35223,2021-04-24T11:13:43.251Z
172,funfair,fun,0.031541,0.324878,173,0.191066,-83.51425,2018-01-08T00:00:00.000Z,0.001055,2885.47675,2020-03-13T02:22:42.216Z
173,redfox-labs-2,rfox,0.240269,0.323790,174,0.377778,-36.58826,2021-04-09T12:20:41.898Z,0.011521,1979.22027,2020-10-28T02:58:18.806Z
...,...,...,...,...,...,...,...,...,...,...,...
2245,velo-token,vlo,0.000489,0.000158,2246,0.058334,-99.16185,2020-12-14T04:20:10.461Z,0.000448,9.02888,2021-04-25T05:59:52.696Z
2246,helix,hlix,0.004828,0.000156,2247,0.037434,-87.10350,2020-05-31T05:45:43.149Z,0.000462,945.56465,2019-07-17T00:00:00.000Z
2247,evil-coin,evil,0.007166,0.000153,2248,0.068920,-89.60175,2018-07-06T03:44:58.098Z,0.000574,1148.80075,2019-10-08T00:00:00.000Z
2248,kittoken,kit,0.000022,0.000151,2249,0.000519,-95.82698,2021-02-18T15:46:36.612Z,0.000003,668.80432,2020-01-08T16:22:42.126Z


In [33]:
obtain_metrics_by_date(left_over_df, coin_data_2018)



Processing alpha-finance
https://api.coingecko.com/api/v3/coins/alpha-finance/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
alpha-finance
https://api.coingecko.com/api/v3/coins/alpha-finance/history?date=01-04-2018localization=False
Processing the-sandbox
https://api.coingecko.com/api/v3/coins/the-sandbox/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
the-sandbox
https://api.coingecko.com/api/v3/coins/the-sandbox/history?date=01-04-2018localization=False
Processing alien-worlds
https://api.coingecko.com/api/v3/coins/alien-worlds/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
alien-worlds
https://api.coingecko.com/api/v3/coins/alien-worlds/history?date=01-04-2018localization=False
Processing funfair
https://api.coingecko.com/api/v3/coins/funfair/history?date=01-04-2018localization=False
Processing redfox-labs-2
https://api.coingecko.com/api/v3/coins/redfox-labs-2/history?da

Key error. Coin too young to find all data
paid-network
https://api.coingecko.com/api/v3/coins/paid-network/history?date=01-04-2018localization=False
Processing math
https://api.coingecko.com/api/v3/coins/math/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
math
https://api.coingecko.com/api/v3/coins/math/history?date=01-04-2018localization=False
Processing hathor
https://api.coingecko.com/api/v3/coins/hathor/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
hathor
https://api.coingecko.com/api/v3/coins/hathor/history?date=01-04-2018localization=False
Processing wanchain
https://api.coingecko.com/api/v3/coins/wanchain/history?date=01-04-2018localization=False
Processing injective-protocol
https://api.coingecko.com/api/v3/coins/injective-protocol/history?date=01-04-2018localization=False
Key error. Coin too young to find all data
injective-protocol
https://api.coingecko.com/api/v3/coins/injective-protocol/history?da

In [34]:
len(coin_data_2018)

227

In [35]:
coins_2018_df = pd.DataFrame(coin_data_2018)
coins_2018_df


,id,price 01-04-2018,reddit_subscribers 01-04-2018,twitter_followers 01-04-2018,alexa_rank 01-04-2018
0,bitcoin,6975.275314,785619.0,819312.0,5574.0
1,ethereum,395.796381,348116.0,382159.0,8204.0
2,binancecoin,11.115511,40605.0,176.0,230.0
3,ripple,0.511990,184063.0,843082.0,5224.0
4,tether,1.000690,NaN,23302.0,34929.0
...,...,...,...,...,...
222,badger-dao,NaN,NaN,NaN,NaN
223,kardiachain,NaN,NaN,NaN,NaN
224,metal,3.544980,NaN,32015.0,153352.0
225,reddcoin,0.003909,38280.0,95656.0,58472.0


In [ ]:
coins_2018_df.to_csv("data/social_data_coins_2018.csv")

